<a href="https://colab.research.google.com/github/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation/blob/main/code_template_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task

We were given InkML files, which contain metadata and a list of strokes:  
[(0,0),(1;0)]...  
These are on-line handwritten mathematical expressions, we'll try to recognize them by using a LG(Labelled Graph) as intermediate output.
The sequence of actions will look like this for now:  
1) Get each stroke  
2) Convert each stroke to symbols (all possible combinations for now we'll optimize later)  
Two strategies available: Consider only consecutive strokes, or allow X(1) number of "time jumps" for each possible symbol.  
2.5) Make a classifier to ignore "junk" stroke combinations, such as a symbol created with every single stroke at the same time.  
3) Classify each constructed symbol with a neural network.

We'll handle spatial relations later  
We'll also handle the final decision later, but we can add a grammar or language model at that point, 

#Environment setup

Please import each file from the git repository to see how they work.  
TODO: link from the git repo, should be feasible.

Hiding my token for obvious reasons.

In [1]:
!git clone https://hiddentoken@github.com/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation.git

Cloning into 'Master_2_MLVC_Recognize_Handwritten_Equation'...
remote: Enumerating objects: 119782, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 119782 (delta 29), reused 58 (delta 18), pack-reused 119711
Receiving objects: 100% (119782/119782), 46.20 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Checking out files: 100% (209487/209487), done.


In [4]:
%cd Master_2_MLVC_Recognize_Handwritten_Equation/code

/content/Master_2_MLVC_Recognize_Handwritten_Equation/code


We will first try out each script then explain what they're supposed to do and how to interpret their output.

## Data to train our classifiers

[This](https://uncloud.univ-nantes.fr/index.php/s/OdAtErZgxKGjsNy) contains a bunch of symbols (in datasymbol_iso/) and expressions (in FullExpressions/) to help train our future classifiers.

#WARNING: training size
there are a LOT of images in the CHROHME dataset, only use a subset to train our classifiers otherwise it'll take too long.

##segmenter.py explanation

In [5]:
!chmod 777 segmenter.py

In [6]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml -o ../data/bidule.lg

In [7]:
!python3 segmenter.py -i ../data/formulaire001-equation001.inkml

O,hyp0,*,1.0,0
O,hyp1,*,1.0,1
O,hyp2,*,1.0,2
O,hyp3,*,1.0,3
O,hyp4,*,1.0,4
O,hyp5,*,1.0,0,1
O,hyp6,*,1.0,1,2
O,hyp7,*,1.0,2,3
O,hyp8,*,1.0,3,4
O,hyp9,*,1.0,0,1,2
O,hyp10,*,1.0,1,2,3
O,hyp11,*,1.0,2,3,4
O,hyp12,*,1.0,0,1,2,3
O,hyp13,*,1.0,1,2,3,4



Each line of this output corresponds to a hypothesis, we can ignore the first four values since they're dummy values indicating a symbol type thing, maybe the position, then the symbol with index, then the symbol without index, then the supposed confidence of our future model(s).. probably, then the strokes used to make that symbol.

For now, we simply list every possible symbol, using every possible stroke in sequence, so if we have 4 strokes, we can make 13 possible symbols.  

segmentSelect.py can be used to remove some hypotheses that don't make sense, for instance, the symbol that could be constructed with stroke [0,1,2,3,4] is highly unlikely to appear, so after training our classifier, we'd remove it.  
Ground truth can be acquired from the lg files of the texts, since we have the strokes used as the 5th...i values.

##segmentSelect.py explanation

In [15]:
!chmod 777 code/segmentSelect.py

In [8]:
!python3 segmentSelect.py -o ../data/bidule2.lg ../data/formulaire001-equation001.inkml ../data/bidule.lg 

In [35]:
!python3 segmentSelect.py ../data/formulaire001-equation001.inkml ../data/bidule.lg 

This is the matrix obtained from convert_to_imgs
<class 'numpy.matrix'>
(32, 32)
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
This is the matrix obtained after converting to tensor
<class 'torch.Tensor'>
torch.Size([1, 32])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
This is the matrix obtained after converting to tensor differently
<class 'torch.Tensor'>
torch.Size([32, 32])
tensor([[255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        ...,
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255],
        [255, 255, 255,  ..., 255, 255, 255]], dtype=torch.uint8)
This is the matrix obtained when using dataloaders
<class 'to

This script takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations alongside the inkml data to generate images, then we check whether these make sense as a symbol, no matter the context.

For now this simply gives a fully random probability that the images are valid symbols.

Need to change two things in there, first adding a classifier to check the validity of supposed symbols,  
then change the threshold based on empirical evidence or what I feel like at the time.

##segmentReco.py explanation

In [10]:
!chmod 777 ./symbolReco.py

In [11]:
!python3 symbolReco.py ../data/formulaire001-equation001.inkml ../data/bidule2.lg

O,hyp0,r,0.059779757207917304,0
O,hyp0,geq,0.05030687013768514,0
O,hyp1,2,0.05648836776140208,1
O,hyp1,6,0.056168315175164935,1
O,hyp1,8,0.05960923199570466,1
O,hyp1,=,0.05475807870783242,1
O,hyp1,geq,0.056816822917386364,1
O,hyp3,f,0.0510902919404698,3
O,hyp3,n,0.0586707731068381,3
O,hyp3,u,0.053721497506933835,3
O,hyp3,3,0.055092772863820916,3
O,hyp4,b,0.05192545736402091,4
O,hyp4,leq,0.07015718890027181,4
O,hyp5,s,0.06121838413470821,0,1
O,hyp5,A,0.050391613253440264,0,1
O,hyp5,X,0.072619141223795,0,1
O,hyp6,(,0.05046390880442578,1,2
O,hyp7,p,0.06723378695795225,2,3
O,hyp7,y,0.05172590875612772,2,3
O,hyp7,1,0.05240988369270244,2,3
O,hyp7,geq,0.058408507485593536,2,3
O,hyp8,Z,0.053352030138174675,3,4
O,hyp9,h,0.051727432180693006,0,1,2
O,hyp9,C,0.05847255666598241,0,1,2
O,hyp9,(,0.05560318869409104,0,1,2
O,hyp9,geq,0.05338792763291579,0,1,2
O,hyp10,r,0.055058494864859676,1,2,3
O,hyp10,4,0.050618596960434595,1,2,3
O,hyp11,c,0.05128946473017116,2,3,4
O,hyp11,j,0.06079852247795053,2,3,4